In [ ]:
import pandas as pd
import numpy as np

In [ ]:
funding_interest_aj = pd.read_csv('../06_Funding_Table_And_Geo_Code/funding_organization_interest.csv')
funding_interest_aj

In [ ]:
update_cite = pd.read_csv('../05_API_Correct_Cite_By_Data/UPDATE_CITE_TITLE.csv').drop(columns="Unnamed: 0")
update_cite

In [ ]:
edit_sub_type = pd.read_csv('../01_Combine_Data/eid&subtype.csv')
edit_sub_type

In [ ]:
funding_interest_aj = funding_interest_aj.merge(edit_sub_type, how='left', on='Eid')
funding_interest_aj

In [ ]:
funding_interest_aj.isna().sum()

In [ ]:
funding_interest_aj = funding_interest_aj[funding_interest_aj['Subtype'] == 'Article']
funding_interest_aj

In [ ]:
funding_interest_aj = funding_interest_aj.merge(update_cite, how='left', on='Eid')
funding_interest_aj

In [ ]:
funding_interest_aj['Subject_Field'] = funding_interest_aj['Subject_Field'].apply(lambda x : str(x)[0:2])
funding_interest_aj = funding_interest_aj.drop_duplicates()
funding_interest_aj

In [ ]:
interest_scrape = pd.read_csv('../10_API_Get_2017_Subject_Area/api_subject_field.csv').drop(columns=['Unnamed: 0'])
interest_scrape

In [ ]:
interest_scrape[interest_scrape['Eid'] == '2-s2.0-85118788235']

In [ ]:
funding_name_list_scrape = pd.read_csv('../09_Table_From_Scraping_Data/funding_list_by_eid_scrape.csv')
funding_name_list_scrape

In [ ]:
interest_scrape = interest_scrape.merge(funding_name_list_scrape, how='left', on='Eid')
interest_scrape

In [ ]:
funding_name_to_country = pd.read_csv('../11_ChatGPT_Funding_Agency_Name_Country/Funding_Name_To_Country_Chat_Clean.csv').drop(columns="Unnamed: 0").rename(columns={"funding_agency_name" : "Agency_Name", "Country" : "Merge_Country"})
funding_name_to_country

In [ ]:
interest_scrape = interest_scrape.merge(funding_name_to_country, how='left', on='Agency_Name')

In [ ]:
interest_scrape['Country'] = np.where(
    interest_scrape['Agency_Country'].notna(), 
    interest_scrape['Agency_Country'], 
    interest_scrape['Merge_Country']
)

In [ ]:
interest_scrape = interest_scrape[~interest_scrape['Country'].isna() & ~interest_scrape['Agency_Name'].isna()][['Eid', 'Agency_Name', 'Subject_Field', 'Country']]
interest_scrape['Year'] = '2017'

In [ ]:
interest_scrape

In [ ]:
funding_interest_scrape = interest_scrape

In [ ]:
update_cite

In [ ]:
funding_interest_scrape = funding_interest_scrape.merge(update_cite, how='left', on='Eid')

In [ ]:
funding_interest_scrape

In [ ]:
funding_interest_aj = funding_interest_aj.rename(columns={"Funding_Agency_Name" : "Agency_Name"})
funding_interest_aj

In [ ]:
country_code_to_name = pd.read_csv('../07_Scrape_Funding_Agency_From_Geo_Link/country_code_name.csv')
country_code_to_name

In [ ]:
funding_interest_aj = funding_interest_aj.merge(country_code_to_name, how='left', on='Funding_Agency_Country_Code')
funding_interest_aj

In [ ]:
funding_interest_aj.isna().sum()

In [ ]:
funding_interest_aj = funding_interest_aj.merge(funding_name_to_country, how='left', on='Agency_Name')

In [ ]:
funding_interest_aj

In [ ]:
funding_interest_aj['Country'] = funding_interest_aj['Country'] = np.where(
    funding_interest_aj['Funding_Agency_Country_Name'].notna(), 
    funding_interest_aj['Funding_Agency_Country_Name'], 
    funding_interest_aj['Merge_Country']
)

In [ ]:
funding_interest_aj = funding_interest_aj[['Eid', 'Agency_Name', 'Subject_Field', 'Publish_month', 'Year', 'Country', 'Title']]
funding_interest_aj

In [ ]:
subject_area_lookup = {
    "10": "Multidisciplinary",
    "11": "Agricultural and Biological Sciences",
    "12": "Arts and Humanities",
    "13": "Biochemistry, Genetics and Molecular Biology",
    "14": "Business, Management and Accounting",
    "15": "Chemical Engineering",
    "16": "Chemistry",
    "17": "Computer Science",
    "18": "Decision Sciences",
    "19": "Earth and Planetary Sciences",
    "20": "Economics, Econometrics and Finance",
    "21": "Energy",
    "22": "Engineering",
    "23": "Environmental Science",
    "24": "Immunology and Microbiology",
    "25": "Materials Science",
    "26": "Mathematics",
    "27": "Medicine",
    "28": "Neuroscience",
    "29": "Nursing",
    "30": "Pharmacology, Toxicology and Pharmaceutics",
    "31": "Physics and Astronomy",
    "32": "Psychology",
    "33": "Social Sciences",
    "34": "Veterinary",
    "35": "Dentistry",
    "36": "Health Professions",
}


In [ ]:
funding_interest_aj['Subject_Field'] = funding_interest_aj['Subject_Field'].apply(lambda x : subject_area_lookup[str(x)])

In [ ]:
funding_interest_aj

In [ ]:
funding_interest_aj.isna().sum()

In [ ]:
funding_interest_aj = funding_interest_aj.dropna().drop_duplicates()

In [ ]:
funding_interest_aj

In [ ]:
funding_interest_scrape = funding_interest_scrape[['Eid', 'Agency_Name', 'Subject_Field', 'Country', 'Title']]

In [ ]:
funding_interest_scrape

In [ ]:
scrape_data_pub_date = pd.read_csv('../12_Scrape_Publish_Date_2017_ChatGPT/Scrape_Data_Pub_Date.csv').rename(columns={'Public_Month' : 'Publish_month', 'Public_Year' : 'Year'})
scrape_data_pub_date.pop('Unnamed: 0')
scrape_data_pub_date

In [ ]:
funding_interest_scrape = funding_interest_scrape.merge(scrape_data_pub_date, how='left', on='Eid')
funding_interest_scrape

In [ ]:
funding_interest_scrape.isna().sum()

In [ ]:
funding_interest_scrape = funding_interest_scrape[funding_interest_scrape['Publish_month'] != 0]

In [ ]:
all_funding_interest = pd.concat([funding_interest_aj, funding_interest_scrape])

In [ ]:
all_funding_interest

In [ ]:
all_funding_interest.isna().sum()

In [ ]:
# all_funding_interest[all_funding_interest['Year'] == '2017']['Eid'].drop_duplicates().to_csv('Pub_Date_2017_Scrape_List.csv')

In [ ]:
funding_country = all_funding_interest['Country'].drop_duplicates().to_list()
for i in funding_country:
    print(i)

In [ ]:
all_funding_interest.isna().sum()

In [ ]:
all_funding_interest.to_csv('TABLE1.csv', index=False)